# 화장품 리뷰 분석
- 네이버 카페 '파우더룸'에서 화장품 리뷰를 크롤링
- 화장품 리뷰들이 카테고리별로 나눠져 있었음 -> 그 중 6개의 카테고리를 크롤링함
- 카테고리 분석, 토픽 분석, 비슷한 단어 분석 등등 다양한 분석을 진행해 볼 것
- (감정분석도 하고 싶었으나 모두 상업용도로 긍정적인 리뷰를 써서 부정을 학습하지 못해 진행하지 못했음)

In [12]:
from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException

In [9]:
import requests

In [1]:
import pandas as pd

In [10]:
driver = Chrome()

In [11]:
url = 'https://cafe.naver.com/cosmania'
driver.get(url)

## 1. 크롤링

### 스킨케어

In [5]:
# 스킨케어 url
skin = driver.find_element_by_css_selector('#menuLink31')
print(skin.get_attribute('href'))
skinurl = skin.get_attribute('href')

https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=31&search.boardtype=L


In [6]:
driver.get(skinurl)

In [7]:
driver.switch_to.frame('cafe_main')
#네이버 카페 게시판은 iframe으로 운영
#iframe 구조로 된 게시판 내부 태그에 접근하기 위해서는 switch_to_frame이 필요

In [8]:
pages = driver.find_elements_by_css_selector('div.prev-next > a')
urls = []

for page in pages:
    urls.append(page.get_attribute('href'))
urls
# url을 보면 맨 뒤에 search.page가 바뀜을 확인할 수 있음

['https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=31&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=1',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=31&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=2',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=31&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=3',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=31&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=4',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=31&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=5',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=31&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=6',
 'https://cafe.naver.com/ArticleList.nhn?searc

In [ ]:
title_urls = []

for i in range(50):
    url = 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=31&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page='+str(i)
    driver.get(url) #페이지 접속
    driver.switch_to.frame('cafe_main')
    
    titles = driver.find_elements_by_css_selector('div.inner_list > a.article') #게시글로 접속하는 태그들
    
    for title in titles:
        if 'referrerAllArticles=false' in title.get_attribute('href'): 
        # 맨 첫 페이지에 이벤트 글이 공지되어있었음 -> 제외하기 위해 if문
            title_urls.append(title.get_attribute('href'))
#title_urls

In [130]:
contents_list = []

for title_url in title_urls:
    driver.get(title_url) #게시글 접속
    driver.switch_to.frame('cafe_main')
    try:
        contents = driver.find_elements_by_css_selector('#tbody > div') #게시글 본문
        title = driver.find_element_by_css_selector('span.b.m-tcol-c').text #게시글 제목
        con = ' '.join([ content.text for content in contents ])
        contents_list.append((title, con,"스킨케어"))
    except NoSuchElementException: #예외의 경우 ex.블로그링크만 올림, 동영상만 올림
        pass
len(contents_list)

750

In [134]:
## 스킨케어 저장
import pandas as pd

df1 = pd.DataFrame(contents_list, columns=['제목', '본문','카테고리'])
df1.head()

,제목,본문,카테고리
0,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법 잇님들 안녕하세요:-...,스킨케어
1,"폴라초이스 스킨 리커버리 클렌저, 모이스처라이저",,스킨케어
2,설인아 콜라겐 아이크림으로 눈가주름 관리해요!,LAVIDA\nCOLLAGEN SOLUTION\nEYE CREAM 올해도 이제...,스킨케어
3,환절기피부관리를 위한 라띠에라 로즈킹덤 세럼,전성분 EWG그린 등급성분으로 만들어진\n기능성 자연화장품 라띠에라 세럼 안녕하세...,스킨케어
4,"유기농화장품 캐롤프리스트 진정세럼, 트러블관리에 딱!이에요.",,스킨케어


In [ ]:
df1.to_excel('pro_skin.xlsx', header=True)

In [42]:
df1 = pd.read_csv("pro_skin.csv",  index_col=0)
df1.head()

,제목,본문,카테고리
0,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법 잇님들 안녕하세요:-...,스킨케어
1,"폴라초이스 스킨 리커버리 클렌저, 모이스처라이저",NaN,스킨케어
2,설인아 콜라겐 아이크림으로 눈가주름 관리해요!,LAVIDA\nCOLLAGEN SOLUTION\nEYE CREAM 올해도 이제...,스킨케어
3,환절기피부관리를 위한 라띠에라 로즈킹덤 세럼,전성분 EWG그린 등급성분으로 만들어진\n기능성 자연화장품 라띠에라 세럼 안녕하세...,스킨케어
4,"유기농화장품 캐롤프리스트 진정세럼, 트러블관리에 딱!이에요.",NaN,스킨케어


### 클렌징

In [145]:
url = 'https://cafe.naver.com/cosmania'
driver.get(url)

In [146]:
# 클렌징 url
cleansing = driver.find_element_by_css_selector('#menuLink39')
cleanurl = cleansing.get_attribute('href')

In [147]:
driver.get(cleanurl)
driver.switch_to.frame('cafe_main')

In [148]:
pages = driver.find_elements_by_css_selector('div.prev-next > a')
urls = []

for page in pages:
    urls.append(page.get_attribute('href'))
urls
# 스킨케어의 url에서 search.menuid만 바뀜을 확인할 수 있다.

['https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=39&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=1',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=39&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=2',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=39&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=3',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=39&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=4',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=39&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=5',
 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=39&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page=6',
 'https://cafe.naver.com/ArticleList.nhn?searc

In [ ]:
title_urls = []

for i in range(50):
    url = 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=39&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page='+str(i)
    driver.get(url) #페이지 접속
    driver.switch_to.frame('cafe_main')
    
    titles = driver.find_elements_by_css_selector('div.inner_list > a.article') #게시글로 접속하는 태그들
    
    for title in titles:
        if 'referrerAllArticles=false' in title.get_attribute('href'): 
        # 맨 첫 페이지에 이벤트 글이 공지되어있었음 -> 제외하기 위해 if문
            title_urls.append(title.get_attribute('href'))
#title_urls

In [160]:
#contents_list2=[]
for title_url in title_urls[369:]:
    driver.get(title_url) #게시글 접속
    driver.switch_to.frame('cafe_main')
    try:
        contents = driver.find_elements_by_css_selector('#tbody > div') #게시글 본문
        title = driver.find_element_by_css_selector('span.b.m-tcol-c').text #게시글 제목
        con = ' '.join([ content.text for content in contents ])
        contents_list2.append((title, con,"클렌징"))
    except NoSuchElementException: #예외의 경우 ex.블로그링크만 올림, 동영상만 올림
        pass
len(contents_list2)

750

In [162]:
## 클렌징 저장
df2 = pd.DataFrame(contents_list2, columns=['제목', '본문','카테고리'])
df2.head()

,제목,본문,카테고리
0,딥클렌징 맑고 깨끗한 피부를 위해!,세안 후에 피부를 보면 오돌토돌 좁쌀부터 \n추워지고 나서부터 피부가 더욱 예민해지...,클렌징
1,클렌징오일추천 뉴트로지나 딥클린 워터프루프도 문제없음♬,항상 립앤아이 리무버로 눈만 대충 지우고\n바로 클렌징폼으로 슥슥 삭삭 얼굴을 닦아...,클렌징
2,"세정력 좋고 순한 클렌징 오일 추천, <스킨큐어 순한 동백 클렌징 오일> 솔직 후기.",안녕하세요. 욱킹스에요.\n미세먼지가 심하죠.\n이럴 때일수록 세안이 중요한 거 다...,클렌징
3,"비페스타 클렌저 3종, 글로우픽1위 수상기념 1+1으로 겟 하자","안녕하세요 잇님들!!\n라미쏭입니댜.\n\n짙은 메이크업부터 데일리 메이크업까지,\...",클렌징
4,순한 클렌징오일 세컨디자인 퍼스트 클렌징오일,#세컨디자인\n#퍼스트클렌징오일 퓨어 앤 프레쉬 듀오\n(블랙/화이트)\n￦ ...,클렌징


In [ ]:
df2.to_excel('pro_cleansing.xlsx', header=True)

In [41]:
df2 = pd.read_csv("pro_cleansing.csv", index_col=0)
df2.head()

,제목,본문,카테고리
0,딥클렌징 맑고 깨끗한 피부를 위해!,세안 후에 피부를 보면 오돌토돌 좁쌀부터 \n추워지고 나서부터 피부가 더욱 예민해지...,클렌징
1,클렌징오일추천 뉴트로지나 딥클린 워터프루프도 문제없음♬,항상 립앤아이 리무버로 눈만 대충 지우고\n바로 클렌징폼으로 슥슥 삭삭 얼굴을 닦아...,클렌징
2,"세정력 좋고 순한 클렌징 오일 추천, <스킨큐어 순한 동백 클렌징 오일> 솔직 후기.",안녕하세요. 욱킹스에요.\n미세먼지가 심하죠.\n이럴 때일수록 세안이 중요한 거 다...,클렌징
3,"비페스타 클렌저 3종, 글로우픽1위 수상기념 1+1으로 겟 하자","안녕하세요 잇님들!!\n라미쏭입니댜.\n\n짙은 메이크업부터 데일리 메이크업까지,\...",클렌징
4,순한 클렌징오일 세컨디자인 퍼스트 클렌징오일,#세컨디자인\n#퍼스트클렌징오일 퓨어 앤 프레쉬 듀오\n(블랙/화이트)\n￦ ...,클렌징


### 선케어

In [166]:
url = 'https://cafe.naver.com/cosmania'
driver.get(url)

In [167]:
# 선케어 url
sun = driver.find_element_by_css_selector('#menuLink42')
sunurl = sun.get_attribute('href')

In [168]:
driver.get(sunurl)
driver.switch_to.frame('cafe_main')

In [ ]:
title_urls = []

for i in range(50):
    url = 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=42&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page='+str(i)
    driver.get(url) #페이지 접속
    driver.switch_to.frame('cafe_main')
    
    titles = driver.find_elements_by_css_selector('div.inner_list > a.article') #게시글로 접속하는 태그들
    
    for title in titles:
        if 'referrerAllArticles=false' in title.get_attribute('href'): 
        # 맨 첫 페이지에 이벤트 글이 공지되어있었음 -> 제외하기 위해 if문
            title_urls.append(title.get_attribute('href'))
#title_urls

In [170]:
contents_list3 = []
for title_url in title_urls:
    driver.get(title_url) #게시글 접속
    driver.switch_to.frame('cafe_main')
    try:
        contents = driver.find_elements_by_css_selector('#tbody > div') #게시글 본문
        title = driver.find_element_by_css_selector('span.b.m-tcol-c').text #게시글 제목
        con = ' '.join([ content.text for content in contents ])
        contents_list3.append((title, con,"선케어"))
    except NoSuchElementException: #예외의 경우 ex.블로그링크만 올림, 동영상만 올림
        pass
len(contents_list3)

750

In [171]:
## 선케어 저장
df3 = pd.DataFrame(contents_list3, columns=['제목', '본문','카테고리'])
df3.head()

,제목,본문,카테고리
0,화장발 업 시키는 이지듀 이지썬 벨벳,정말 화장하고 다니기 힘든 요즘\n민낯에 자신 있으신 분들은 하나만 바르면 ...,선케어
1,제가 오랫동안 쓰고 있는 선크림이에요~,"저는 선크림을 고를 때 기준이 있어요\n1, 바를때 살짝 하얘지는 느낌2, 촉촉하게...",선케어
2,데일리 썬크림 추천! 백탁현상없는 순한 썬크림~,SNS입소문난 블랙아웃 썬크림을 만나봤어요.\n뷰티방송에서도 봐서 어떨지 ...,선케어
3,고급스러움과 촉촉함이 한번에 담긴 발드르 썬스틱~,고급스러움과 촉촉함이 한번에 담긴 발드르 썬스틱~ 뷰스타 바리에오 !\n오늘은 발...,선케어
4,이지듀 이지썬 벨벳프라이머 하나면 매끈한 깐달걀피부 어렵지 않아요:D,"지난 7월부터 줄곧- 열심히 사용해 오고 있는\n이지듀 이지썬 벨벳프라이머, 벌써 ...",선케어


In [ ]:
df3.to_excel('pro_sun.xlsx', header=True)

In [40]:
df3 = pd.read_csv('pro_sun.csv', index_col=0)
df3.head()

,제목,본문,카테고리
0,화장발 업 시키는 이지듀 이지썬 벨벳,정말 화장하고 다니기 힘든 요즘\n민낯에 자신 있으신 분들은 하나만 바르면 ...,선케어
1,제가 오랫동안 쓰고 있는 선크림이에요~,"저는 선크림을 고를 때 기준이 있어요\n1, 바를때 살짝 하얘지는 느낌2, 촉촉하게...",선케어
2,데일리 썬크림 추천! 백탁현상없는 순한 썬크림~,SNS입소문난 블랙아웃 썬크림을 만나봤어요.\n뷰티방송에서도 봐서 어떨지 ...,선케어
3,고급스러움과 촉촉함이 한번에 담긴 발드르 썬스틱~,고급스러움과 촉촉함이 한번에 담긴 발드르 썬스틱~ 뷰스타 바리에오 !\n오늘은 발...,선케어
4,이지듀 이지썬 벨벳프라이머 하나면 매끈한 깐달걀피부 어렵지 않아요:D,"지난 7월부터 줄곧- 열심히 사용해 오고 있는\n이지듀 이지썬 벨벳프라이머, 벌써 ...",선케어


### 베이스 메이크업

In [174]:
url = 'https://cafe.naver.com/cosmania'
driver.get(url)

In [5]:
# 베이스 url
base = driver.find_element_by_css_selector('#menuLink34')
baseurl = base.get_attribute('href')

In [6]:
driver.get(baseurl)
driver.switch_to.frame('cafe_main')

In [ ]:
title_urls = []

for i in range(50):
    url = 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=34&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page='+str(i)
    driver.get(url) #페이지 접속
    driver.switch_to.frame('cafe_main')
    
    titles = driver.find_elements_by_css_selector('div.inner_list > a.article') #게시글로 접속하는 태그들
    
    for title in titles:
        if 'referrerAllArticles=false' in title.get_attribute('href'): 
        # 맨 첫 페이지에 이벤트 글이 공지되어있었음 -> 제외하기 위해 if문
            title_urls.append(title.get_attribute('href'))
title_urls

In [49]:
contents_list4 = []
for title_url in title_urls:
    driver.get(title_url) #게시글 접속
    driver.switch_to.frame('cafe_main')
    try:
        contents = driver.find_elements_by_css_selector('#tbody > div') #게시글 본문
        title = driver.find_element_by_css_selector('span.b.m-tcol-c').text #게시글 제목
        con = ' '.join([ content.text for content in contents ])
        contents_list4.append((title, con,"베이스메이크업"))
    except NoSuchElementException: #예외의 경우 ex.블로그링크만 올림, 동영상만 올림
        pass

352

In [50]:
## 베이스메이크업 저장
df4 = pd.DataFrame(contents_list4, columns=['제목', '본문','카테고리'])
df4.head()

,제목,본문,카테고리
0,[더블유랩] 은은한 광채의 쥬얼리베이스 루비 / 팔자주름 없애주는 프라이머 베이스 ...,안녕하세요 채홍이에요 ღ˘⌣˘ღ\n오늘 소개해드릴 잇템은\n요즘같은 환절기에 쉽게...,베이스메이크업
1,[아페리레] 데이 드림 커버 쿠션(핑크 쿠션)/ 핑크 덕후들 모여라~~(A쿠션 커버...,안녕하세요 채홍입니다\n오늘 소개해드릴 핑크쿠션!!\n비쥬얼 정말 장난아닙니다.....,베이스메이크업
2,[홀리카홀리카] 가을대비하여 나온 글로우파운데이션과 스키니 아이브로우 리뷰,,베이스메이크업
3,"조르지오 아르마니 파운데이션 ""루미너스 실크"", 무너짐이 예쁜 파운데이션.",조르지오 아르마니는\n매장이 많이 없어\n저한테는 더욱 소장욕이 커요.\n\n\n...,베이스메이크업
4,유튜버추천 ! 메이블린 핏미 컨실러 15호 페어,유튜버추천 ! 메이블린 핏미 컨실러 15호 페어\n\n\n안녕하세요!\n뷰스타 치니...,베이스메이크업


In [51]:
df4.to_excel('pro_base.xlsx', header=True)

In [39]:
df4 = pd.read_csv('pro_base.csv', encoding='utf8',  index_col=0)
df4.head()

,제목,본문,카테고리
0,올리브영 11월 세일 팩트 추천 루나 에센스 수분광 팩트 FX FW 리미티드 에디션,안녕하세여 여러분\n뷰스타 류꽃분입니다!\n\n제가 이번 포스팅에서 소개드릴 제품...,베이스메이크업
1,겨울 베이스 대란템~잉가 66듀드롭 파운데이션 & 에센스 팩트 비교,NaN,베이스메이크업
2,오샐런스 :: 오샐런스 톤업베이스 - 촉촉한 톤업베이스,"오샐런스 톤업베이스\n30g, 24,000원\n안녕하세요!\n오늘 소개해 드릴 제품...",베이스메이크업
3,페리페라 2018 홀리데이 ASMR 컬렉션 발색샷!,페리페라 2018 홀리데이 컬렉션 소개하려구 달려와써욥 ! (∗❛⌄❛∗)\n페...,베이스메이크업
4,[톤업선크림추천] 오샐런스 톤업베이스,\n\n\n톤업선크림 오샐런스 톤업베이스는요?!\n\n자외선 차단 SPF50+ P...,베이스메이크업


### 아이 메이크업

In [52]:
url = 'https://cafe.naver.com/cosmania'
driver.get(url)

In [54]:
# 아이메이크업 url
eye = driver.find_element_by_css_selector('#menuLink35')
eyeurl = eye.get_attribute('href')

In [55]:
driver.get(eyeurl)
driver.switch_to.frame('cafe_main')

In [ ]:
title_urls = []

for i in range(50):
    url = 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=35&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page='+str(i)
    driver.get(url) #페이지 접속
    driver.switch_to.frame('cafe_main')
    
    titles = driver.find_elements_by_css_selector('div.inner_list > a.article') #게시글로 접속하는 태그들
    
    for title in titles:
        if 'referrerAllArticles=false' in title.get_attribute('href'): 
        # 맨 첫 페이지에 이벤트 글이 공지되어있었음 -> 제외하기 위해 if문
            title_urls.append(title.get_attribute('href'))
#title_urls

In [ ]:
contents_list5 = []
for title_url in title_urls:
    driver.get(title_url) #게시글 접속
    driver.switch_to.frame('cafe_main')
    try:
        contents = driver.find_elements_by_css_selector('#tbody > div') #게시글 본문
        title = driver.find_element_by_css_selector('span.b.m-tcol-c').text #게시글 제목
        con = ' '.join([ content.text for content in contents ])
        contents_list5.append((title, con,"아이메이크업"))
    except NoSuchElementException: #예외의 경우 ex.블로그링크만 올림, 동영상만 올림
        pass
#contents_list5

In [ ]:
## 아이메이크업 저장
df5 = pd.DataFrame(contents_list5, columns=['제목', '본문','카테고리'])
df5.head()

In [ ]:
df5.to_excel('pro_eye.xlsx', header=True)

In [37]:
df5 = pd.read_csv('pro_eye.csv', index_col=0)
df5.head()

,제목,본문,카테고리
0,3CE 멀티 아이 컬러 팔레트 오버테이크. 발색 및 웜톤 쿨톤 아이메이크업.,,아이메이크업
1,그윽한 브라운마스카라 추천해!! 릴리바이레드 나인투나인 서바이벌 컬러마스카라 5종,안녕하세요 뷰티블로거 셔니예요!\n\n오늘은 그윽장전템 #브라운마스카라 5종을 가...,아이메이크업
2,연말 화려한 펄감의 아이메이크업::이글립스 라이블리 리퀴드 글리터 섀도우 6종,안녕하세요 굥블리입니다❣️\n오늘은 화려한 펄감에 날림&무너짐이 없는\n이글립스...,아이메이크업
3,이글립스 라이블리 리퀴드 글리터 섀도우 전색상발색! 연말 메이크업으로 찰떡 인것같아...,안녕하세요 나루루에요\n오늘은 썸네일 보고 놀라셨죠..?\n달덩이 같은 제 얼굴이...,아이메이크업
4,이글립스 라이블리 리퀴드 글리터 섀도우 전색상 발색♬ 연말 글리터 메이크업해봤어요!,안녕하세요 뷰스타 어나더에요★\n\n\n연말이 다가오면서 약속 자리가 많아지죠*_...,아이메이크업


### 립 메이크업

In [14]:
# 립메이크업 url
lip = driver.find_element_by_css_selector('#menuLink36')
lipurl = lip.get_attribute('href')

In [15]:
driver.get(lipurl)
driver.switch_to.frame('cafe_main')

In [17]:
title_urls = []

for i in range(50):
    url = 'https://cafe.naver.com/ArticleList.nhn?search.clubid=10050813&search.menuid=36&search.boardtype=L&search.questionTab=A&search.totalCount=151&search.page='+str(i)
    driver.get(url) #페이지 접속
    driver.switch_to.frame('cafe_main')
    
    titles = driver.find_elements_by_css_selector('div.inner_list > a.article') #게시글로 접속하는 태그들
    
    for title in titles:
        if 'referrerAllArticles=false' in title.get_attribute('href'): 
        # 맨 첫 페이지에 이벤트 글이 공지되어있었음 -> 제외하기 위해 if문
            title_urls.append(title.get_attribute('href'))

In [27]:
#contents_list6 = []
for title_url in title_urls[350:]:
    driver.get(title_url) #게시글 접속
    driver.switch_to.frame('cafe_main')
    try:
        contents = driver.find_elements_by_css_selector('#tbody > div') #게시글 본문
        title = driver.find_element_by_css_selector('span.b.m-tcol-c').text #게시글 제목
        con = ' '.join([ content.text for content in contents ])
        contents_list6.append((title, con,"립메이크업"))
    except NoSuchElementException: #예외의 경우 ex.블로그링크만 올림, 동영상만 올림
        pass
len(contents_list6)

750

In [28]:
## 립메이크업 저장
df6 = pd.DataFrame(contents_list6, columns=['제목', '본문','카테고리'])
df6.head()

,제목,본문,카테고리
0,슈에무라 X 라 메종 뒤 쇼콜라 홀리데이 컬렉션 ★ 달달함의 최고봉 컬렉션으로 인정 !,안녕하세요 여러분 뷰스타 미미입니다 : )\n\n일단 오늘은 저같은 초퀄릿 ~.~...,립메이크업
1,에스쁘아 노웨어 메가벨벳 립스틱 끝장 오버립 ♥,"에스쁘아 노웨어 메가벨벳 립스틱 끝장 오버립 ♥\n가격: 23,000원 용량: ...",립메이크업
2,컬러그램 벨벳 마그넷틴트 8호 내일도픽&9호 무드플라쓰와 힛팬 섀도우 1호 팥팥으로...,컬러그램 벨벳 마그넷틴트 8호 내일도픽&9호 무드플라쓰와\n힛팬 섀도우 1호 팥팥...,립메이크업
3,"이글립스 벨벳 핏 틴트 5종, 색감이 미쳤어요..♡",안녕하세요 뷰티블로거 셔니예요:)\n\n이글립스에서 넘나리 이쁜 #벨벳틴트 5종이...,립메이크업
4,"페리페라 ASMR 잉크 더 에어리 벨벳, 슈가 캔디 틴트 스틱, 슈가 젤리 틴트",안녕하세요. 플뢰르예요.\n페리페라 ASMR 립 제품을 보여드릴게요.\n페리페라 ...,립메이크업


In [35]:
df6.to_excel('pro_lip.xlsx', header=True)

In [36]:
df6 = pd.read_csv('pro_lip.csv', encoding='utf8', index_col=0)
df6.head()

,제목,본문,카테고리
0,슈에무라 X 라 메종 뒤 쇼콜라 홀리데이 컬렉션 ★ 달달함의 최고봉 컬렉션으로 인정 !,안녕하세요 여러분 뷰스타 미미입니다 : )\n\n일단 오늘은 저같은 초퀄릿 ~.~...,립메이크업
1,에스쁘아 노웨어 메가벨벳 립스틱 끝장 오버립 ♥,"에스쁘아 노웨어 메가벨벳 립스틱 끝장 오버립 ♥\n가격: 23,000원 용량: ...",립메이크업
2,컬러그램 벨벳 마그넷틴트 8호 내일도픽&9호 무드플라쓰와 힛팬 섀도우 1호 팥팥으로...,컬러그램 벨벳 마그넷틴트 8호 내일도픽&9호 무드플라쓰와\n힛팬 섀도우 1호 팥팥...,립메이크업
3,"이글립스 벨벳 핏 틴트 5종, 색감이 미쳤어요..♡",안녕하세요 뷰티블로거 셔니예요:)\n\n이글립스에서 넘나리 이쁜 #벨벳틴트 5종이...,립메이크업
4,"페리페라 ASMR 잉크 더 에어리 벨벳, 슈가 캔디 틴트 스틱, 슈가 젤리 틴트",안녕하세요. 플뢰르예요.\n페리페라 ASMR 립 제품을 보여드릴게요.\n페리페라 ...,립메이크업


### 최종파일

In [2]:
import pandas as pd

In [ ]:
total=pd.concat([df1,df2,df3,df4,df5,df6], ignore_index=True)
total.to_csv("pro_total.csv")

In [3]:
total = pd.read_csv("pro_total.csv", index_col=0)
total.head()

,제목,본문,카테고리
0,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법 잇님들 안녕하세요:-...,스킨케어
1,"폴라초이스 스킨 리커버리 클렌저, 모이스처라이저",NaN,스킨케어
2,설인아 콜라겐 아이크림으로 눈가주름 관리해요!,LAVIDA\r\nCOLLAGEN SOLUTION\r\nEYE CREAM 올해...,스킨케어
3,환절기피부관리를 위한 라띠에라 로즈킹덤 세럼,전성분 EWG그린 등급성분으로 만들어진\r\n기능성 자연화장품 라띠에라 세럼 안녕...,스킨케어
4,"유기농화장품 캐롤프리스트 진정세럼, 트러블관리에 딱!이에요.",NaN,스킨케어


In [4]:
tot = total.dropna(axis=0).reset_index()
del tot["index"]
tot.head()

,제목,본문,카테고리
0,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법 잇님들 안녕하세요:-...,스킨케어
1,설인아 콜라겐 아이크림으로 눈가주름 관리해요!,LAVIDA\r\nCOLLAGEN SOLUTION\r\nEYE CREAM 올해...,스킨케어
2,환절기피부관리를 위한 라띠에라 로즈킹덤 세럼,전성분 EWG그린 등급성분으로 만들어진\r\n기능성 자연화장품 라띠에라 세럼 안녕...,스킨케어
3,프레쥬 오리지날 허브 자연약쑥 진정 수분크림,겨울이 되어 피부가 많이 건조해졌는데 수분크림을 찾게 되는데 자연약쑥과 7가지 그린...,스킨케어
4,위드미 스노우 화이트닝 포어 토어,화장을 하고나면 입주변에 여드름이 올라와 고민이었어요.\r\nT존으로 피지가 있는 ...,스킨케어


## 2. 형태소 분석

In [5]:
from worin.tag import Worin

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
tagger= Worin()
def get_nouns(text):
    try: # 에러가 발생하지 않을 경우
        nouns = tagger.nouns(text)
        nouns = [word for word in nouns if len(word) > 1]
        return nouns
    except: # 에러 발생
        return ["error"]

In [7]:
## 본문내용이 없는 데이터 삭제
# 본문에 내용이 없는 데이터가 있었음
# 제외하기 위해 글자가 없는 경우의 정규표현식 -> 데이터 인덱스 추출 -> 인덱스 삭제하기
import re
novalue = []

for i in range(len(tot)):
    if re.findall('[a-zA-Z0-9가-힣]+', tot.iloc[i,1]) == []: #글자가 없는 경우의 인덱스
        novalue.append(i)
    else:
        continue

In [8]:
len(novalue)

57

In [9]:
# 데이터 삭제
tot_no = tot.drop(novalue,axis=0)
len(tot_no)

4138

## 3. TDM

### 3.1 TDM

In [360]:
## CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, tokenizer=get_nouns)

In [361]:
## TDM
tdm = cv.fit_transform(tot_no['본문'])

In [376]:
for i, n in sorted(zip(tdm[0].indices, tdm[0].data)):
    print(words[i], n)

가격 2
가득 1
가지 1
건강 1
건조 2
겨울 2
공급 1
관리 1
구매 1
끈적 1
날씨 2
느낌 2
때문 2
민감 2
방법 3
부족 1
브랜드 1
비타민 1
사용 8
사이즈 1
산뜻 1
살짝 1
상품 2
생각 1
성분 2
세안 1
소개 1
손등 1
수량 1
수분 4
수분감 2
수분크림 5
시원 1
아이 1
아침 1
얼굴 1
여드름 1
예민 2
완전 1
요즘 1
용기 1
용량 1
유분기 1
유수분 1
이상 1
장벽 1
저녁 2
제품 1
주의 1
중요 1
진정 7
쫀쫀 1
촉촉 3
추천 1
추천드 1
충전 1
쿨링감 1
크림 4
탄탄 1
판매 1
피부 7
피부관리 1
피부속 1
할인 2
허브 5
홍조 1
화해 1
확인 2
흡수 1


### 3.2 TF-IDF적용

In [362]:
## TF-IDF로 변환
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tdm2 = tfidf.fit_transform(tdm)
words = cv.get_feature_names()

In [364]:
for i, n in sorted(zip(tdm2[0].indices, tdm2[0].data)):
    print(words[i], n)

가격 0.08640343617312517
가득 0.0765354321818814
가지 0.03584252505434694
건강 0.05568995535316314
건조 0.07479065558255643
겨울 0.10405571898246149
공급 0.07479990375322082
관리 0.057490970309387324
구매 0.04500523260735404
끈적 0.06328392576878093
날씨 0.11256602623014257
느낌 0.05379255171542367
때문 0.07938093327298064
민감 0.09718593710448545
방법 0.20587134881587776
부족 0.07354929371876862
브랜드 0.05009804099990995
비타민 0.07072385396099617
사용 0.1746445925584784
사이즈 0.0625159849665282
산뜻 0.07999032763623193
살짝 0.07053872592651247
상품 0.1629767600115831
생각 0.03632788637017956
성분 0.07212967801364639
세안 0.04940983021786719
소개 0.04006273500611399
손등 0.04833752260033949
수량 0.08604025415444436
수분 0.1818846028751986
수분감 0.10042150658654538
수분크림 0.32432276297764523
시원 0.06427867121692131
아이 0.04297739255053023
아침 0.06070779862262429
얼굴 0.03599041493709599
여드름 0.0750373798123218
예민 0.11489574912786155
완전 0.05199647071075809
요즘 0.03802444472662009
용기 0.06086455304072362
용량 0.044880322207057836
유분기 0.06402454496004557
유수분 0.0

### 3.3 저장 후 파일 열기

In [11]:
from sklearn.externals import joblib

In [369]:
## 저장
from sklearn.externals import joblib
with open('pro_tdm2.pkl', 'wb') as f:
    joblib.dump({'vectorizer': cv, 'tdm': tdm, 'tfidf': tdm2, 'category':tot_no['카테고리']}, f)

In [12]:
with open('pro_tdm2.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)

### 3.4 단어빈도표

In [371]:
## 단어빈도표
wordcount = pd.DataFrame(
    {'단어': cv.get_feature_names(),
     '빈도': tdm2.sum(axis=0).flat
    }
)
wordcount.head()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:271: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


,단어,빈도
0,가격,47.781466
1,가능,23.631173
2,가득,9.127629
3,가루날림,12.106061
4,가벼,9.110571


In [375]:
wordcount.sort_values('빈도', ascending=False).head()

,단어,빈도
926,피부,319.132383
815,컬러,296.612901
367,사용,284.855462
731,제품,229.992919
843,클렌징,194.378052


## 4. 주제분석

### 4.1 문서클러스터링
- 스펙트럴 클러스터링
- k-means 클러스터링
- 정규화 후, k-means 클러스터링

여러 클러스터링 방법을 진행하여 각 클러스터 당 어떤 단어가 많이 나오는지 본 후, 주관적으로 클러스터를 카테고리 이름으로 바꾸었다. 후에 원래 데이터의 카테고리와 예측된 클러스터의 label끼리의 정확도를 보았다.

[사이트에 나온 기준]
- 스킨케어: 스킨, 에센스, 크림, 앰플
- 클렌징: 클렌징 제품, 각질제거 제품
- 선케어: 자외선차단제
- 베이스메이크업: 파운데이션, 파우더, 비비, 하이라이터, 블러셔
- 아이메이크업: 아이라이너, 마스카라, 아이섀도우
- 립메이크업: 립스틱, 립글로스, 틴드

#### 4.1.1 스펙트럴 클러스터링: 유클리드 거리

In [386]:
from sklearn.cluster import SpectralClustering
cl = SpectralClustering(n_clusters=6, random_state=1234)
labels = cl.fit_predict(tdm2[0:])

In [401]:
from operator import itemgetter
words = vectorizer.get_feature_names()

def top20(labels):
    freq_words = []
    for i in range(6):
        count = tdm[labels == i, :].sum(axis=0)
        ws = [w for w, n in sorted(zip(words, count.flat), key=itemgetter(1), reverse=True)[:20]]
        freq_words.append(ws)

    return pd.DataFrame(freq_words)

In [402]:
top20(labels)
# 0: 스킨케어
# 1: 립메이크업
# 2: 베이스메이크업
# 3: 선케어
# 4: 아이메이크업
# 5: 클렌징

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,피부,사용,제품,촉촉,느낌,크림,성분,메이크업,건조,제형,수분,이지듀,얼굴,컬러,파운데이션,효과,베이스,앰플,자외선,오일
1,컬러,발색,느낌,사용,입술,제품,립스틱,색상,틴트,메이크업,핑크,촉촉,레드,가지,가을,벨벳,매트,오렌지,블러,생각
2,쿠션,페리페라,사용,피부,컬러,제품,핑크,촉촉,컬렉션,잉크,느낌,커버력,발색,색상,순간,메이크업,커버,파운데이션,리뷰,베이지
3,선크림,피부,사용,제품,자외선,촉촉,차단,크림,이지듀,이지,느낌,성분,무기자차,백탁,톤업,추천,백탁현상,제형,얼굴,메이크업
4,이너,아이,마스카라,사용,컬러,제품,메이크업,속눈썹,브라운,블랙,색상,페리페라,느낌,핑크,발색,리퀴드,눈매,잉크,워터프루프,라인
5,클렌징,피부,사용,제품,메이크업,세안,오일,촉촉,거품,자극,성분,느낌,클렌징폼,이지듀,노폐물,워터,부드,세정력,렌징,얼굴


In [413]:
labels_list = []
for i in range(len(labels)):
    if labels[i] == 0:
        labels_list.append('스킨케어')
    elif labels[i] == 1:
        labels_list.append('립메이크업')
    elif labels[i] == 2:
        labels_list.append('베이스메이크업')
    elif labels[i] == 3:
        labels_list.append('선케어')
    elif labels[i] == 4:
        labels_list.append('아이메이크업')
    elif labels[i] == 5:
        labels_list.append('클렌징')

In [422]:
from sklearn.metrics import accuracy_score
accuracy_score(labels_list, category)

0.5923151280811987

#### 4.1.2 k-means클러스터링

In [392]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=6, random_state=1234)
labels_km = km.fit_predict(tdm2)

In [423]:
top20(labels_km)
# 0: 클렌징
# 1: 스킨케어
# 2: 아이메이크업
# 3: 베이스메이크업
# 4: 선케어
# 5: 립메이크업

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,클렌징,피부,사용,제품,세안,메이크업,오일,촉촉,거품,자극,성분,느낌,클렌징폼,이지듀,부드,노폐물,세정력,렌징,워터,얼굴
1,피부,사용,제품,촉촉,크림,느낌,성분,건조,파운데이션,수분,메이크업,제형,앰플,얼굴,오일,효과,때문,팩트,추천,요즘
2,컬러,사용,메이크업,제품,아이,발색,이너,느낌,마스카라,색상,브라운,핑크,글리터,가지,페리페라,속눈썹,팔레트,블랙,베이스,블러
3,쿠션,피부,사용,제품,촉촉,느낌,컬러,핑크,커버력,메이크업,퍼프,페리페라,컬렉션,커버,베이스,건조,광채,화사,베이지,지속력
4,선크림,피부,제품,사용,자외선,촉촉,이지듀,이지,차단,크림,성분,느낌,톤업,메이크업,프라이머,베이스,백탁,얼굴,여름,효과
5,컬러,발색,입술,느낌,틴트,립스틱,색상,제품,핑크,사용,촉촉,레드,벨벳,페리페라,가을,잉크,가지,착색,매트,이션


In [427]:
labels_km_list = []
for i in range(len(labels_km)):
    if labels_km[i] == 0:
        labels_km_list.append('클렌징')
    elif labels_km[i] == 1:
        labels_km_list.append('스킨케어')
    elif labels_km[i] == 2:
        labels_km_list.append('아이메이크업')
    elif labels_km[i] == 3:
        labels_km_list.append('베이스메이크업')
    elif labels_km[i] == 4:
        labels_km_list.append('선케어')
    elif labels_km[i] == 5:
        labels_km_list.append('립메이크업')

In [428]:
accuracy_score(labels_km_list, category)

0.7414209763170614

#### 4.1.3 정규화 후 K-Means 클러스터링

In [394]:
from sklearn.preprocessing import Normalizer
nom = Normalizer(copy=False)
pos = nom.fit_transform(tdm2)
km = KMeans(n_clusters=6, random_state=1234)
labels_nom = km.fit_predict(pos)

In [429]:
top20(labels_nom)
# 0: 클렌징
# 1: 스킨케어
# 2: 아이메이크업
# 3: 베이스메이크업
# 4: 선케어
# 5: 립메이크업

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,클렌징,피부,사용,제품,세안,메이크업,오일,촉촉,거품,자극,성분,느낌,클렌징폼,이지듀,부드,노폐물,세정력,렌징,얼굴,워터
1,피부,사용,제품,촉촉,크림,느낌,성분,건조,파운데이션,수분,메이크업,제형,앰플,얼굴,오일,효과,때문,팩트,추천,요즘
2,컬러,사용,메이크업,제품,아이,발색,이너,느낌,마스카라,색상,브라운,핑크,글리터,가지,페리페라,속눈썹,팔레트,블랙,베이스,블러
3,쿠션,피부,사용,제품,촉촉,느낌,컬러,핑크,커버력,메이크업,퍼프,페리페라,컬렉션,커버,베이스,건조,광채,화사,베이지,지속력
4,선크림,피부,제품,사용,자외선,촉촉,이지듀,이지,차단,크림,성분,느낌,톤업,메이크업,프라이머,베이스,백탁,얼굴,여름,효과
5,컬러,발색,입술,느낌,틴트,립스틱,색상,제품,핑크,사용,촉촉,레드,벨벳,페리페라,가을,잉크,가지,착색,매트,이션


In [430]:
labels_nom_list = []
for i in range(len(labels_nom)):
    if labels_nom[i] == 0:
        labels_nom_list.append('클렌징')
    elif labels_nom[i] == 1:
        labels_nom_list.append('스킨케어')
    elif labels_nom[i] == 2:
        labels_nom_list.append('아이메이크업')
    elif labels_nom[i] == 3:
        labels_nom_list.append('베이스메이크업')
    elif labels_nom[i] == 4:
        labels_nom_list.append('선케어')
    elif labels_nom[i] == 5:
        labels_nom_list.append('립메이크업')

In [431]:
accuracy_score(labels_nom_list, category)

0.7426292895118415

-> 클러스터링을 한 결과, 스펙트럴클러스터링<k-means<정규화 후 k-means 순으로 정확도가 높았다.

### 4.2 LDA
- 스킨케어, 클렌징, 선케어, 베이스메이크업, 아이메이크업, 립메이크업마다 다양한 화장품 제품과 그에 대한 다양한 평가가  있을 것
-> LDA를 통해 각 문서의 주제 비율들을 확인해 볼 것이다.

In [492]:
words = vectorizer.get_feature_names()
word_dict = dict(enumerate(words))
word_dict[0]

'가격'

In [491]:
## gensim형식으로 변환
from gensim.matutils import Sparse2Corpus
corpus = Sparse2Corpus(tfidf.T)

In [519]:
from gensim.models.ldamodel import LdaModel
lda = LdaModel(corpus=corpus,
               num_topics=200,
               passes=3,
               iterations=100,
               id2word=word_dict,
               random_state=123)

C:\Users\user\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [520]:
## 첫 번째 문서 단어 빈도수 확인
row = tfidf[0]
doc = list(zip(row.indices, row.data))
doc_words = [(words[i], n) for i, n in doc]
doc_words
# 첫 번째 문서: 겨울철 건조할 때, 혹은 피부 진정용으로 사용하는 수분크림

[('방법', 0.20587134881587776),
 ('완전', 0.05199647071075809),
 ('겨울', 0.10405571898246149),
 ('날씨', 0.11256602623014257),
 ('수분', 0.1818846028751986),
 ('피부속', 0.08391202465096992),
 ('건조', 0.07479065558255643),
 ('피부관리', 0.07720025382662184),
 ('사용', 0.1746445925584784),
 ('피부', 0.18118117967205344),
 ('소개', 0.04006273500611399),
 ('허브', 0.47157187248885407),
 ('진정', 0.38955794623132434),
 ('수분크림', 0.32432276297764523),
 ('용량', 0.044880322207057836),
 ('가격', 0.08640343617312517),
 ('부족', 0.07354929371876862),
 ('유분기', 0.06402454496004557),
 ('크림', 0.1775582020316117),
 ('여드름', 0.0750373798123218),
 ('유수분', 0.08332167854277316),
 ('촉촉', 0.08370937837087813),
 ('건강', 0.05568995535316314),
 ('관리', 0.057490970309387324),
 ('아이', 0.04297739255053023),
 ('중요', 0.05184037951175266),
 ('생각', 0.03632788637017956),
 ('성분', 0.07212967801364639),
 ('화해', 0.07999032763623193),
 ('확인', 0.10157131233330532),
 ('주의', 0.0801504845243595),
 ('때문', 0.07938093327298064),
 ('추천', 0.03704965187478513),
 ('예민

In [521]:
## 첫 번째 문서의 주제 비율
lda.get_document_topics(doc)

[(18, 0.18035927),
 (46, 0.13274622),
 (61, 0.13422942),
 (129, 0.0115427775),
 (137, 0.010770471),
 (161, 0.06465253),
 (192, 0.3358892)]

In [522]:
lda.show_topic(192) #피부를 위한 제품

[('시크', 0.04490991),
 ('피부', 0.042103413),
 ('팩트', 0.034939725),
 ('에센스', 0.032440037),
 ('크림', 0.020385234),
 ('수분', 0.017531903),
 ('자외선', 0.016472084),
 ('세럼', 0.016248576),
 ('미백', 0.0149337845),
 ('사용', 0.014886775)]

In [528]:
lda.save('pro_lda.lda')
lda2 = LdaModel.load('pro_lda.lda')

## 5. 지도학습

### 5.1 로지스틱 회귀로 분류

In [17]:
## 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf, category, test_size=.2, random_state=1234)

In [586]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class='multinomial', solver='saga')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [587]:
model.score(X_train, y_train)

0.9540785498489426

In [588]:
model.score(X_test, y_test)

0.9142512077294686

In [638]:
## 회귀계수
categories = [
    '립메이크업',
    '베이스메이크업',
    '선케어',
    '스킨케어',
    '아이메이크업',
    '클렌징'
] #ㄱㄴㄷ순

import pandas as pd
weights = pd.DataFrame(model.coef_.T)
weights.index = cv.get_feature_names()
weights.columns = categories

top5 = []
for category in categories:
    top5.append(weights[category].sort_values(ascending=False).head().reset_index())
pd.concat(top5, axis=1)

,index,립메이크업,index,베이스메이크업,index,선케어,index,스킨케어,index,아이메이크업,index,클렌징
0,입술,4.841310,쿠션,4.264787,선크림,5.649150,피부,2.953989,마스카라,3.324824,클렌징,5.494624
1,립스틱,3.998284,파운데이션,3.591700,자외선,3.948380,크림,2.711599,글리터,2.775007,거품,2.682264
2,틴트,3.776878,블러,3.269589,선스틱,3.141115,오일,2.293520,브라운,2.642562,세안,2.574471
3,컬러,2.808623,톤업크림,2.853944,썬크림,2.729545,토너,2.027467,이너,2.614264,약산,2.082864
4,착색,2.011389,커버력,2.744912,차단,2.199510,앰플,1.965074,속눈썹,2.392716,필링젤,2.048432


### 5.2  keras 다층 신경망으로 분류

In [13]:
from keras import Sequential
from keras.layers import Dense, Input

Using TensorFlow backend.


In [14]:
## 카테고리를 숫자로 변환
tot_num = tot_no.copy()
tot_num['카테고리'] = pd.factorize(tot_no['카테고리'])[0]
tot_num.head()

,제목,본문,카테고리
0,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법,수부지수분크림 프레쥬 자연약쑥 진정크림으로 피부좋아지는 방법 잇님들 안녕하세요:-...,0
1,설인아 콜라겐 아이크림으로 눈가주름 관리해요!,LAVIDA\r\nCOLLAGEN SOLUTION\r\nEYE CREAM 올해...,0
2,환절기피부관리를 위한 라띠에라 로즈킹덤 세럼,전성분 EWG그린 등급성분으로 만들어진\r\n기능성 자연화장품 라띠에라 세럼 안녕...,0
3,프레쥬 오리지날 허브 자연약쑥 진정 수분크림,겨울이 되어 피부가 많이 건조해졌는데 수분크림을 찾게 되는데 자연약쑥과 7가지 그린...,0
4,위드미 스노우 화이트닝 포어 토어,화장을 하고나면 입주변에 여드름이 올라와 고민이었어요.\r\nT존으로 피지가 있는 ...,0


In [614]:
# 저장
with open('pro_tdm3.pkl', 'wb') as f:
    joblib.dump({'vectorizer': cv, 'tdm': tdm, 'tfidf': tdm2, 'category':tot_num['카테고리']}, f)

In [15]:
# 열기
with open('pro_tdm3.pkl', 'rb') as f:
    data2 = joblib.load(f)
locals().update(data2)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, category, test_size=.2, random_state=1234)

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

In [22]:
NUM_WORDS = X_train.shape[1]

In [23]:
## 모형 생성
nn = Sequential()
nn.add(Dense(256, activation='relu', input_shape=(NUM_WORDS,)))
nn.add(Dropout(rate=0.5))
nn.add(Dense(6, activation='softmax'))

In [24]:
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 256)               256256    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 1542      
Total params: 257,798
Trainable params: 257,798
Non-trainable params: 0
_________________________________________________________________


In [25]:
## 학습
nn.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
nn.fit(X_train, y_train, epochs=30)

Epoch 1/30
3310/3310 [==============================] - 5s 2ms/step - loss: 1.1864 - acc: 0.7804A: 1s - loss: 1.3248 - 
Epoch 2/30
3310/3310 [==============================] - 3s 947us/step - loss: 0.3424 - acc: 0.9284
Epoch 3/30
3310/3310 [==============================] - 3s 827us/step - loss: 0.2123 - acc: 0.9396
Epoch 4/30
3310/3310 [==============================] - 3s 821us/step - loss: 0.1695 - acc: 0.9523
Epoch 5/30
3310/3310 [==============================] - 3s 844us/step - loss: 0.1410 - acc: 0.9577
Epoch 6/30
3310/3310 [==============================] - 3s 945us/step - loss: 0.1215 - acc: 0.9650
Epoch 7/30
3310/3310 [==============================] - 3s 882us/step - loss: 0.1038 - acc: 0.9695
Epoch 8/30
3310/3310 [==============================] - 3s 1ms/step - loss: 0.0941 - acc: 0.9683
Epoch 9/30
3310/3310 [==============================] - 3s 949us/step - loss: 0.0836 - acc: 0.9743
Epoch 10/30
3310/3310 [==============================] - 3s 860us/step - loss: 0.0814 - ac

In [26]:
## 예측
y_pred = nn.predict_classes(X_test)

In [28]:
## 평가
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9178743961352657

### 5.3 LIME

In [29]:
from lime.lime_text import LimeTextExplainer

In [31]:
categories = [
    '스킨케어',
    '클렌징',
    '선케어',
    '베이스메이크업',
    '아이메이크업',
    '립메이크업',
]

In [32]:
exp = LimeTextExplainer(class_names=categories)

In [663]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer()

In [156]:
def predict_fn(text):
    mat = tfidf[idx]
    return nn.predict(mat)

In [157]:
idx=500
print(tot_num.iloc[idx,2])
predict_fn([tot_num.iloc[idx,1]])
#0번째 주제, 즉 스킨케어의 주제임을 0.99999094로 예측하였다. -> 잘 예측

0


array([[9.9999094e-01, 5.9617523e-06, 1.8093805e-06, 8.5117517e-07,
        6.8666473e-08, 3.6178568e-07]], dtype=float32)

In [159]:
idx=4000
print(tot_num.iloc[idx,2])
predict_fn([tot_num.iloc[idx,1]])
#5번째 주제, 즉 스킨케어의 주제임을 0.9999994로 예측하였다. -> 잘 예측

5


array([[6.5260025e-08, 4.1193456e-08, 4.0372825e-08, 3.4342438e-07,
        9.4456389e-08, 9.9999940e-01]], dtype=float32)

In [161]:
idx=3000
print(tot_num.iloc[idx,2])
predict_fn([tot_num.iloc[idx,1]])
#4번째 주제, 즉 스킨케어의 주제임을 0.999999642로 예측하였다. -> 잘 예측

4


array([[9.60444835e-10, 1.14010845e-07, 6.91641633e-09, 1.14248017e-07,
        9.99999642e-01, 9.92022891e-08]], dtype=float32)

## 7. 워드임베딩

In [42]:
import re

In [48]:
## 정규표현식
words = tot_no.iloc[:,1]
total_words = []

for word in words:
    total_words.append(word)
result = ''.join(total_words)

In [61]:
result2 = re.sub(r'[^a-zA-Z가-힣\. ]', ' ' ,result)

In [62]:
sentences = result2.split('. ')  # 문장 단위로 자름

In [57]:
word = [s.split() for s in sentences] #단어 단위로 자름

In [58]:
word[0] #단어 단위가 아니라서 worin형태소 분석기를 사용할 것임

['수부지수분크림',
 '프레쥬',
 '자연약쑥',
 '진정크림으로',
 '피부좋아지는',
 '방법',
 '잇님들',
 '안녕하세요',
 '얼마',
 '전',
 '눈도',
 '펑펑내리고',
 '완전하게',
 '겨울이라해도',
 '되는',
 '날씨에요',
 '이런날씨에',
 '수분부족형지성이신분들은',
 '피부속은',
 '건조한데',
 '기름기는',
 '올라오고',
 '피부관리하시기가',
 '엄청',
 '어렵잖아요',
 '요때',
 '제가',
 '매일',
 '사용하는',
 '크림과',
 '요크림을',
 '사용해서',
 '피부',
 '좋아지는',
 '방법들을',
 '소개해볼께요',
 '프레쥬',
 '오리지날',
 '허브',
 '자연약쑥',
 '진정',
 '수분크림',
 '용량',
 'ml',
 '가격',
 '원',
 '수분이',
 '부족한',
 '지성피부이신분들은',
 '겨울이라고해서',
 '무작정',
 '유분기',
 '많은',
 '크림을',
 '사용하다보면',
 '얼굴에',
 '여드름올라오고',
 '기름기에',
 '더',
 '감당하기',
 '힘들어짐',
 '이때',
 '유수분을',
 '잘맞춰줘야지',
 '진짜',
 '촉촉하고',
 '건강한',
 '피부로',
 '관리할',
 '수',
 '있어요',
 '그래서',
 '요',
 '아이는',
 '물광이',
 '촉촉하게',
 '만들어주는',
 '진정수분크림이에요',
 '제가',
 '중요하게',
 '생각하는',
 '것',
 '중',
 '하나인',
 '성분을',
 '화해어플을',
 '통해',
 '확인해보면',
 '주의성분은',
 '물론이고',
 '알레르기주의성분이',
 '도',
 '없기',
 '때문에',
 '착한성분의',
 '크림으로',
 '추천',
 '예민하신',
 '분들이나',
 '민감하신분들도',
 '사용하기',
 '좋은',
 '제품이에요',
 '비타민이랑',
 '미네랄이',
 '많이',
 '있는',
 '자연약쑥',
 '가지와',
 '그린허브',
 '성분이',
 '함유되어있어서',
 '피부',
 '안쪽까지',
 '수분을

In [63]:
## Worin 형태소 분석기 사용
tagger= Worin()
def get_nouns(text):
    try: # 에러가 발생하지 않을 경우
        nouns = tagger.nouns(text)
        nouns = [word for word in nouns if len(word) > 1]
        return nouns
    except: # 에러 발생
        return ["error"]

In [71]:
documents = []
for i in range(len(tot_no.iloc[:,1])):
    documents.append(get_nouns(tot_no.iloc[i,1]))

In [72]:
documents[0]

['수부지수분크림',
 '프레쥬',
 '자연약',
 '진정크림',
 '방법',
 '눈도',
 '내리',
 '완전',
 '겨울',
 '날씨',
 '날씨',
 '수분',
 '족형지성',
 '피부속',
 '건조',
 '기름기',
 '피부관리',
 '사용',
 '요크림',
 '사용',
 '피부',
 '방법',
 '소개',
 '프레쥬',
 '오리지날',
 '허브',
 '자연약',
 '진정',
 '수분크림',
 '용량',
 '가격',
 '수분',
 '부족',
 '지성피부이',
 '겨울',
 '무작정',
 '유분기',
 '크림',
 '사용',
 '여드름',
 '기름기',
 '감당',
 '이때',
 '유수분',
 '촉촉',
 '건강',
 '피부',
 '관리',
 '아이',
 '물광이',
 '촉촉',
 '진정수분크림',
 '중요',
 '생각',
 '하나인',
 '성분',
 '화해',
 '확인',
 '주의',
 '레르기주',
 '때문',
 '크림',
 '추천',
 '예민',
 '민감',
 '사용',
 '제품',
 '비타민',
 '미네',
 '자연약',
 '가지',
 '성분',
 '피부',
 '수분',
 '공급',
 '쑥이',
 '예민',
 '피부',
 '진정',
 '정화',
 '장벽',
 '탄탄',
 '크림',
 '초록초촉',
 '느낌',
 '사용',
 '용기',
 '스파',
 '츌러',
 '화장대',
 '스파',
 '츌러',
 '사용',
 '사용',
 '수분감',
 '가득',
 '젤리느낌',
 '수딩제형',
 '촉촉',
 '끈적',
 '산뜻',
 '쿨링감',
 '시원',
 '느낌',
 '요즘',
 '손등',
 '거칠거',
 '크림',
 '피부',
 '흡수',
 '수분감',
 '쫀쫀',
 '차갑',
 '건조',
 '겨울바람',
 '민감',
 '홍조',
 '수분기',
 '피부',
 '방법중',
 '냉장',
 '프레쥬크림',
 '팩처',
 '얼굴',
 '진정',
 '수분',
 '충전',
 '방법',
 '아침',
 '살짝',
 '때문',
 '저녁',
 '사용',
 '저녁',
 '세안

### 7.1 word2vec

In [73]:
from gensim.models.word2vec import Word2Vec

C:\Users\user\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [74]:
model = Word2Vec(documents,         # 리스트 형태
                 sg=1,         # 0: CBOW, 1: Skip-gram
                 size=100,     # 벡터 크기
                 window=3,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=3,  # 사용할 단어의 최소 빈도(3회 이하 단어 무시)
                 workers=4)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)

In [75]:
## 저장
model.save('pro_word2vec.model')

In [147]:
## 불러오기
model = Word2Vec.load('pro_word2vec.model')

In [77]:
model.wv['촉촉']

array([ 0.00206848,  0.19459786, -0.36315572, -0.25976145, -0.32418898,
       -0.1259948 , -0.15635905,  0.5630231 ,  0.31470937, -0.4078463 ,
       -0.07118844,  0.07123663,  0.5127987 ,  0.03773263,  0.16502243,
        0.19521718, -0.76599735, -0.13801402,  0.4299486 , -0.08641271,
        0.09360606, -0.05941821,  0.22162907,  0.0522872 ,  0.01017583,
        0.21776785, -0.1921876 ,  0.2109211 , -0.33357105, -0.17369644,
        0.10658416, -0.43604293,  0.19019349,  0.06696663, -0.23186733,
        0.07476347, -0.25658235,  0.46853906,  0.21824913, -0.08589438,
       -0.04941929,  0.5435351 ,  0.25557834, -0.35975823, -0.74260265,
        0.02078794, -0.1007634 , -0.1443807 , -0.16007172, -0.24756561,
       -0.02646115,  0.64817697,  0.13426745,  0.3508993 , -0.30741987,
       -0.24422365,  0.12260474, -0.5627442 , -0.51213217,  0.15569426,
       -0.54504746,  0.13294935,  0.03852362, -0.34569603, -0.22761475,
        0.05731123,  0.00822555,  0.08291311,  0.00224289,  0.54

In [91]:
## 코사인 유사도
model.wv.similarity('컬러', '레드')

0.7306014281904194

In [93]:
## 코사인 유사도
model.wv.similarity('촉촉', '수분크림')

0.6645324541927546

In [90]:
## 가장 유사한 단어
model.wv.most_similar('물광')

[('무결점', 0.8230122327804565),
 ('물광피부', 0.8000475764274597),
 ('프리미어', 0.7957494258880615),
 ('광채', 0.7916102409362793),
 ('수분팩', 0.7900696992874146),
 ('광채피부', 0.789462685585022),
 ('쿨링', 0.7868645787239075),
 ('부스팅', 0.7861065864562988),
 ('충전', 0.7829052209854126),
 ('선사', 0.7815368175506592)]

In [79]:
## 가장 유사한 단어
model.wv.most_similar('촉촉')

[('흡수력', 0.8149070739746094),
 ('초크초크', 0.8045000433921814),
 ('리치', 0.7973361015319824),
 ('무겁', 0.7933701276779175),
 ('습감', 0.7858553528785706),
 ('뽀송', 0.7838507294654846),
 ('워터리', 0.7833592891693115),
 ('보습감', 0.7820384502410889),
 ('초경량', 0.7810738682746887),
 ('수분감', 0.7803847193717957)]

In [81]:
## 가장 유사한 단어
model.wv.most_similar('애굣살')

[('덩이', 0.9713246822357178),
 ('중앙', 0.9695633053779602),
 ('눈두덩이', 0.9569394588470459),
 ('애교살', 0.9547940492630005),
 ('눈두덩', 0.9529669284820557),
 ('운데', 0.9472670555114746),
 ('영역', 0.9457536339759827),
 ('눈앞머리', 0.9432390332221985),
 ('앞머리', 0.9362964630126953),
 ('입체감', 0.9281405210494995)]

In [87]:
## 가장 유사한 단어
model.wv.most_similar('밀착')

[('고밀착', 0.9015926122665405),
 ('매끄', 0.8888841271400452),
 ('착력', 0.8739922642707825),
 ('매끈', 0.8541513681411743),
 ('주름사', 0.8504502177238464),
 ('초경량', 0.8481572270393372),
 ('루날림', 0.8476483821868896),
 ('픽싱력', 0.8450065851211548),
 ('발리', 0.8441821336746216),
 ('밀착감', 0.8432092666625977)]

In [148]:
## 가장 유사한 단어
model.wv.most_similar('틴트')

[('벨벳틴트', 0.6983211636543274),
 ('틴트인', 0.6824373602867126),
 ('립제품', 0.6727991700172424),
 ('과즙팡', 0.6665595769882202),
 ('과즙팡틴트', 0.6641845703125),
 ('립틴트', 0.6457366943359375),
 ('입술', 0.6416869163513184),
 ('미샤', 0.6305449604988098),
 ('어퓨', 0.6287322044372559),
 ('라커틴트', 0.6279722452163696)]